## 01 Validação cruzada e aleatoriedade inicial

### 01.02 A influência da aleatoriedade na validação do modelo

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

preco  vendido  idade_do_modelo   km_por_ano
0   30941.02        1               18  35085.22134
1   40557.96        1               20  12622.05362
2   89627.50        0               12  11440.79806
3   95276.14        0                3  43167.32682
4  117384.68        1                4  12770.11290

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y)
print(f"Treinaremos com {treino_x.__len__()} e testaremos com {teste_x.__len__()} elementos")


Treinaremos com 7500 e testaremos com 2500 elementos


In [3]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
accuracy = dummy_stratified.score(teste_x, teste_y) * 100

print(f"A acurácia do dummy_stratified foi {accuracy:.2f}")

A acurácia do dummy_stratified foi 58.00


In [4]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
model = DecisionTreeClassifier(max_depth=2)
model.fit(treino_x, treino_y)
predictions = model.predict(teste_x)

accuracy = accuracy_score(teste_y, predictions) * 100
print(f"A acurácia foi {accuracy:.2f}")

A acurácia foi 71.92


In [5]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y)
print(f"Treinaremos com {treino_x.__len__()} e testaremos com {teste_x.__len__()} elementos")

model = DecisionTreeClassifier(max_depth=2)
model.fit(treino_x, treino_y)
predictions = model.predict(teste_x)

accuracy = accuracy_score(teste_y, predictions) * 100
print(f"A acurácia foi {accuracy:.2f}")

Treinaremos com 7500 e testaremos com 2500 elementos
A acurácia foi 76.84


### 01.04 A validação cruzada

#### Treino/Teste (holdout): 0.70 treino + 0.30 teste -> O resultado varia com cada proporção de treino/teste

#### Validação cruzada (cross validation/k-fold): Divide-se a base em $k$ pedaços, treina-se e testa-se $k$ vezes e calcula a média dos resultados.

#### Teste de hipótese(?) -> Resultado em intervalo de probabilidade.

### 01.06 Usando e avaliando com o cross validate

[sklearn.model_selection.cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)

In [6]:
from sklearn.model_selection import cross_validate

SEED = 158020
# SEED = 301
# SEED = 5
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=3) # cv: default value if None is 5-fold
mean = results['test_score'].mean()
standard_deviation = results['test_score'].std()

print(f"Accuracy with cross validation, 3 = [{(mean - 2 * standard_deviation) * 100:.2f}, {(mean + 2 * standard_deviation) * 100:.2f}]")

Accuracy with cross validation, 3 = [74.99, 76.57]


In [8]:
SEED = 301
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=5) # cv: default value if None is 5-fold # deterministic
mean = results['test_score'].mean()
standard_deviation = results['test_score'].std()

print(f"Accuracy with cross validation, 5 = [{(mean - 2 * standard_deviation) * 100:.2f}, {(mean + 2 * standard_deviation) * 100:.2f}]")

Accuracy with cross validation, 5 = [75.21, 76.35]


## 02 KFold e aleatoriedade

### 02.02 Kfold com aleatorização

In [15]:
def print_results(results):
    mean = results['test_score'].mean()
    standard_deviation = results['test_score'].std()
    print(f"Accuracy mean: {(mean * 100):.2f}")
    print(f"Accuracy interval: [{(mean - 2 * standard_deviation) * 100:.2f}, {(mean + 2 * standard_deviation) * 100:.2f}]")

In [17]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=cv) # cv: default value if None is 5-fold # deterministic
print_results(results)

Accuracy mean: 75.78
Accuracy interval: [74.37, 77.19]


In [19]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True) # shuffling data before split
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=cv) # cv: default value if None is 5-fold # deterministic
print_results(results)

Accuracy mean: 75.76
Accuracy interval: [73.26, 78.26]


## 03. Estratificação

### 03.02 Estratificação com validação cruzada

#### Simular uma situação horrível de azar:

Pode ser "azar" como pode ser uma proporção de exemplos desbalanceados entre as classes.

In [26]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

preco  vendido  idade_do_modelo   km_por_ano
55  76666.68        0               16  24846.60026
41  57463.74        0               15  23114.95042
42  41296.05        0               10  19485.88872
43  96446.05        0               17  38476.10072
45  64387.60        0               10  23609.01780

In [28]:
# sem embaralhar
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_azar, y_azar, cv=cv) # cv: default value if None is 5-fold # deterministic
print_results(results)

Accuracy mean: 57.94
Accuracy interval: [34.47, 81.41]


In [29]:
# embaralhando
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_azar, y_azar, cv=cv) # cv: default value if None is 5-fold # deterministic
print_results(results)

Accuracy mean: 75.78
Accuracy interval: [74.45, 77.11]


In [30]:
# estratificado
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_azar, y_azar, cv=cv) # cv: default value if None is 5-fold # deterministic
print_results(results)

Accuracy mean: 75.78
Accuracy interval: [73.45, 78.11]
